In [ ]:
from src.data_reader import DataReader
from src.constant import *
import matplotlib.pyplot as plt
import numpy as np
from src.datasets.dataset import Dataset
from src.models.itemknn.itemknn import ItemKNN
import pandas as pd
from src.evaluation import compute_mrr

In [ ]:
dataset = Dataset()
split_dict = dataset.get_split()
train, train_label = split_dict[TRAIN]

In [ ]:
train_data = dataset.get_train_sessions()
lead_data = dataset.get_test_leaderboard_sessions()
test_data = dataset.get_test_final_sessions()
all_data = pd.concat([train_data, lead_data, test_data], axis=0)

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
rand1 = torch.rand((10, 2))
norm_rand1 = F.normalize(rand1)
rand2 = torch.rand((10, 2))
torch.einsum("bf,bf->b", rand1, rand2)

In [ ]:
norm_rand1

In [ ]:
rand1

In [ ]:
rand.view(1024,-1,64).shape

In [ ]:
prova = torch.Tensor(np.array([[1,2,3], [0,0,1]]))
prova

In [ ]:
prova.mean(dim=-1)

In [ ]:
den = (prova!=0).sum(dim=-1)
num = prova.sum(dim=-1)
num/den
# torch.nonzero(prova)
#mask = prova!=0
#prova*mask.sum(dim=-1)/mask.sum(dim=-1)

In [ ]:
emb = torch.nn.Embedding(4, 10, padding_idx=0)

In [ ]:
emb_tensor = emb.weight

In [ ]:
emb_tensor

In [ ]:
emb_tensor[torch.LongTensor([0,1])]

In [ ]:
torch.mean(emb(torch.LongTensor([0,1,2])))

In [ ]:
torch.mean(emb(torch.LongTensor([1,2])))

In [ ]:
a, b = train_label[[SESS_ID, ITEM_ID]].values[0]

In [ ]:
b

In [ ]:
train[[SESS_ID, ITEM_ID]].to_dict(orient="records")

In [ ]:
dataset = Dataset()

In [ ]:
item_features = dataset.get_item_features()

In [ ]:
item_features

In [ ]:
oh_cat = pd.get_dummies(item_features[F_CAT], prefix = "cat")

In [ ]:
oh_val = pd.get_dummies(item_features[F_VAL], prefix="val")

In [ ]:
item_features_oh = item_features.join(oh_cat).join(oh_val)

In [ ]:
item_features_oh = item_features_oh.groupby(ITEM_ID).sum()

In [ ]:
item_features_oh = item_features_oh.drop([F_VAL, F_CAT], axis=1).reset_index()

In [ ]:
item_features_oh